In [25]:
from random import random
import copy

In [26]:
class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None
    temp_board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns

    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly 

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"
        
        self.__init_board()
        turns_played = 0
        current_player = starter
        while(turns_played < self.board_size):
            
            if (current_player == self.YOU):
                self.__print_board()
                temp_board = copy.deepcopy(self.board)
                player_input = self.get_your_input(copy.deepcopy(self.board),5,True,-100000,100000)[1]
                self.board = copy.deepcopy(temp_board)
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    def get_your_input(self,position, depth, maxPlayer,a,b):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        #TODO: complete here


    # If the game came to an end, the function needs to return
    # the evaluation function of the end. 

    
        self.board = position
        if depth == 0 or self.check_for_winners() != 0:
            return self.evaluate(),0

        if maxPlayer:
            chosen_column = None
            best_value = -100000
            all_child_boards = self.get_all_board(1,position)
            columns = self.get_possible_moves_simulate(position)
            if not all_child_boards:
                return -100000,0
            for child_board in all_child_boards:
                val = self.get_your_input(child_board,depth - 1, False,a,b)[0]
                #val = self.get_your_input(child_board,depth - 1, False)[0]
                best_value = max(best_value, val)
                a = max(a,best_value)
                if best_value == val:
                    chosen_column = columns[all_child_boards.index(child_board)]
                if b <= a:
                    break
            return best_value , chosen_column

        else:
            chosen_column = None
            best_value = 100000
            all_child_boards = self.get_all_board(-1,position)
            columns = self.get_possible_moves_simulate(position)
            if not all_child_boards:
                return 100000,0
            for child_board in all_child_boards:
                val = self.get_your_input(child_board,depth - 1, True,a,b)[0]
                #val = self.get_your_input(child_board,depth - 1, True)[0]
                best_value = min(best_value, val)
                b = min(b,best_value)
                if best_value == val:
                    chosen_column = columns[all_child_boards.index(child_board)]
                if b <= a:
                    break
            return best_value , chosen_column

    def evaluate(self):
        """ Simple heuristic to evaluate board configurations
            Heuristic is (num of 4-in-a-rows)*99999 + (num of 3-in-a-rows)*100 + 
            (num of 2-in-a-rows)*10 - (num of opponent 4-in-a-rows)*99999 - (num of opponent
            3-in-a-rows)*100 - (num of opponent 2-in-a-rows)*10
        """
        
        my_fours = self.checkForStreak(self.YOU, 4)
        my_threes = self.checkForStreak(self.YOU, 3)
        my_twos = self.checkForStreak(self.YOU, 2)
        opp_fours = self.checkForStreak(self.CPU, 4)
        opp_threes = self.checkForStreak(self.CPU, 3)
        opp_twos = self.checkForStreak(self.CPU, 2)

        return my_fours*100000 + my_threes*1000 + my_twos*10 - opp_fours*100000 - opp_threes*1000 - opp_twos*10
            
    def checkForStreak(self,player_id, streak):
        count = 0
        c1 = False
        c2 = False 
        c3 = False
        # for each piece in the board...
        for i in range(self.rows):
            for j in range(self.columns):
                # ...that is of the color we're looking for...
                # check if a vertical streak starts at (i, j)
                c1 = self.verticalStreak(player_id, streak)
                    
                # check if a horizontal four-in-a-row starts at (i, j)
                c2 = self.horizontalStreak(player_id, streak)
                    
                # check if a diagonal (either way) four-in-a-row starts at (i, j)
                c3 = self.diagonalCheck(player_id, streak)

                if c1 == True :
                    count +=1
                if c2 == True :
                    count +=1 
                if c3 == True :
                    count +=1
        # return the sum of streaks of length 'streak'
        return count

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )
    
    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0
    
    def is_move_valid_simulate(self, move,board):
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return board[0][column_index] == 0
    
    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves
    
    def get_possible_moves_simulate(self,board):
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid_simulate(move,board)):
                possible_moves.append(move)
        return possible_moves
    
    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True
    

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)
    

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False
    
    def horizontalStreak(self, player_id,streak):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(streak):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def verticalStreak(self, player_id,streak):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(streak):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def diagonalCheck(self, player_id,streak):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(streak):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(streak):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]
    
    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]
    
    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return True
    
    def drop_piece_in_simulate_column(self, move, current_player,board):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (board[i][column_index] == 0):
                last_empty_space = i
        board[last_empty_space][column_index] = current_player
        return board
        
    def get_all_board(self,player,board):
        all_moves = []
        columns = self.get_possible_moves_simulate(board)
        for column in columns:
            if self.is_move_valid(column) :
                simulate_board = copy.deepcopy(board)
                new_board = self.drop_piece_in_simulate_column(column,player,simulate_board)
                all_moves.append(new_board) 
        return all_moves
    

    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")
    
    def __change_turn(self, turn):
        if (turn == self.YOU): 
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"
                    
                print(f"{house_char}", end=" ")
            print()



In [27]:
board_sizes_to_check = [(6,7), 
                        (7,8), 
                        (7,10)]
game = ConnectSin(board_size=(6,7),silent=False)
game.run()

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O O 
O O O C O O Y 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O Y 
C O O C O O Y 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O C O O Y 
O O O C O O Y 
C O O C O O Y 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O Y 
O O O C O O Y 
O O O C O O Y 
C O O C O O Y 
congrats! you have won!


1